<a href="https://www.kaggle.com/code/mikedelong/essay-keywords-first-look?scriptVersionId=153074758" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
! pip install sentence-transformers 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e188fbe87f1d3af849a0efa52ab431081c3094dc5d19843e93a0b24370740c54
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
with open(file='/kaggle/input/paul-graham-essays/paul_graham_essays.txt', encoding='utf-8', mode='r') as input_fp:
    text = input_fp.read()
essays = text.split('\n')
print('essay count: {}'.format(len(essays)))

essay count: 55


In [3]:
# https://www.kaggle.com/code/ianalyticsgeek/keywords-extraction-using-bert
from arrow import now
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# we can't use more features than we can visualize
MAX_FEATURES = 600

NGRAM_RANGE = (1, 1)
model_start = now()
count = TfidfVectorizer(ngram_range=NGRAM_RANGE, stop_words='english', max_features=MAX_FEATURES).fit(essays)
features = count.get_feature_names_out()
print('{}: got features'.format(now()))
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embedding = model.encode(essays)
print('{}: got embeddings'.format(now()))
feature_embeddings = model.encode(features)
print('{}: got feature embeddings'.format(now()))
print('model time: {}'.format(now() - model_start))

2023-11-30T19:52:39.658022+00:00: got features


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2023-11-30T19:52:49.688984+00:00: got embeddings


Batches:   0%|          | 0/19 [00:00<?, ?it/s]

2023-11-30T19:52:51.135771+00:00: got feature embeddings
model time: 0:00:11.767059


In [4]:
# post process the features 
keep = set(features)
# filter out numeric values
keep = {item for item in keep if not item.isnumeric()}
# do some basic plural identification and remove the plurals 
plurals = {item for item in keep if (item.endswith('s') and item[:-1] in keep) or (item.endswith('es') and item[:-2] in keep)}
keep = keep.difference(plurals)
print(len(keep), keep)

534 {'value', 'success', 'multiple', 'happen', 'default', 'growing', 'fund', 'away', 'simply', 'tend', 'valuation', 'possible', 'investment', 'worked', 'field', 'advantage', 'day', 'tv', 'usually', 'look', 'asked', 'history', 'round', 'order', 'ambitious', 'used', 'answer', 'little', 'hardware', 'intelligence', 'said', 'live', 'steve', 'spent', 'huge', 'theory', 'high', 'let', 'create', 'ask', 'fairly', 'matter', 'started', 'people', 'telling', 'interested', 'today', 'plus', 'surprising', 'sure', 'bought', 'built', 'rich', 'dollars', 'sense', 'ago', 'car', 'write', 'site', 'won', 'independent', 'using', 'drafts', 'sign', 'going', 'interesting', 'seed', 'realize', 'large', 'surprised', 'spend', 'close', 'fun', 'come', 'minded', 'book', 'fact', 'painting', 'apple', 'certainly', 'shouldn', 'famous', 'decided', 'truth', 'explain', 'having', 'worth', 'version', 'control', 'trevor', 'longer', 'person', 'friend', 'stop', 'blackwell', 'university', 'dangerous', 'run', 'job', 'hire', 'investing

In [5]:
from nltk import pos_tag
from nltk import word_tokenize

noun_start = now()
keep_nouns = {word for item in essays for (word, part_of_speech) in pos_tag(word_tokenize(item)) if part_of_speech[:2] == 'NN' and word in keep}
print('filtered nouns in {}'.format(now() - noun_start))
keep = keep_nouns
print(len(keep), keep)

filtered nouns in 0:00:24.465440
348 {'value', 'success', 'multiple', 'happen', 'default', 'fund', 'tend', 'valuation', 'investment', 'field', 'advantage', 'day', 'look', 'round', 'history', 'order', 'answer', 'hardware', 'intelligence', 'live', 'spent', 'theory', 'let', 'create', 'matter', 'ask', 'people', 'today', 'surprising', 'plus', 'sure', 'bought', 'rich', 'dollars', 'sense', 'car', 'write', 'site', 'drafts', 'sign', 'interesting', 'seed', 'realize', 'close', 'fun', 'come', 'book', 'fact', 'painting', 'apple', 'famous', 'truth', 'explain', 'worth', 'version', 'control', 'longer', 'person', 'friend', 'stop', 'university', 'run', 'job', 'hire', 'investing', 'filter', 'suspect', 'office', 'code', 'economy', 'hard', 'selling', 'century', 'play', 'idea', 'desktop', 'paul', 'grad', 'hacking', 'business', 'care', 'knowledge', 'principle', 'situation', 'stage', 'ignore', 'founder', 'word', 'like', 'kind', 'human', 'model', 'saying', 'organizations', 'startup', 'raise', 'use', 'trick', '

In [6]:
# now we can project the feature vectors into 2-space to visualize 
from pandas import DataFrame
from plotly.express import scatter
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=2023, verbose=1, n_iter=10000)
tsne_df = DataFrame(data=tsne.fit_transform(X=feature_embeddings), columns=['t0', 't1'])
tsne_df['word'] = features
scatter(data_frame=tsne_df[tsne_df['word'].isin(keep)], x='t0', y='t1', text='word', height=900, ).update_traces(marker={'size': 1})

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 600 samples in 0.001s...
[t-SNE] Computed neighbors for 600 samples in 0.156s...
[t-SNE] Computed conditional probabilities for sample 600 / 600
[t-SNE] Mean sigma: 3.295514
[t-SNE] KL divergence after 250 iterations with early exaggeration: 61.629822
[t-SNE] KL divergence after 3500 iterations: 1.081234
